In [1]:
from model import Model
from util import convert_tokens, get_batch_dataset, get_dataset, get_record_parser
import tensorflow as tf
from config import flags
import numpy as np
import json

In [2]:
flags.DEFINE_string('f', 'give up already', 'who cares lol')
config = flags.FLAGS

In [4]:
test_eval_file = 'data/og_check_test_meta.json'
test_record_file = 'data/og_check_test.tf'

#test_eval_file = 'data/sample1k-HCVerifyAll_eval.json'
#test_record_file = 'data/sample1k-HCVerifyAll.tfrecords'

with open(test_eval_file, "r") as fh:
    eval_file = json.load(fh)

meta = {'total': 1382}

In [5]:
with open(config.word_emb_file, "r") as fh:
    word_mat = np.array(json.load(fh), dtype=np.float32)
with open(config.char_emb_file, "r") as fh:
    char_mat = np.array(json.load(fh), dtype=np.float32)

In [6]:
test_batch = get_dataset(test_record_file, get_record_parser(
        config, is_test=True), config).make_one_shot_iterator()

In [7]:
model = Model(config, test_batch, word_mat, char_mat, trainable=False)

Instructions for updating:
Use the retry module or similar alternatives.


In [8]:
sess_config = tf.ConfigProto(allow_soft_placement=True)
sess_config.gpu_options.allow_growth = True

sess = tf.Session(config=sess_config)

In [9]:
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint(config.save_dir))
sess.run(tf.assign(model.is_train, tf.constant(False, dtype=tf.bool)))

INFO:tensorflow:Restoring parameters from log/model/model_60000.ckpt


False

In [10]:
total = meta['total']
print(total)

1382


In [11]:
answer_dict = {}
remapped_dict = {}

for step in range(total // config.batch_size + 1):
    qa_id, loss, yp1, yp2 = sess.run([model.qa_id, model.loss, model.yp1, model.yp2])
    answer_dict_, remapped_dict_ = convert_tokens(eval_file, qa_id.tolist(), yp1.tolist(), yp2.tolist())
    answer_dict.update(answer_dict_)
    remapped_dict.update(remapped_dict_)

In [12]:
from util import metric_max_over_ground_truths, exact_match_score, f1_score

In [13]:
def evaluate(eval_file, answer_dict, only=None):
    f1 = exact_match = total = 0
    for key, value in answer_dict.items():
        if only == 'adv' and len(eval_file[key]['uuid'].split('-')) == 1:
            continue
        if only == 'orig' and len(eval_file[key]['uuid'].split('-')) > 1:
            continue
        total += 1
        ground_truths = eval_file[key]["answers"]
        prediction = value
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(f1_score,
                                            prediction, ground_truths)
#         if metric_max_over_ground_truths(exact_match_score, prediction, ground_truths) < 1:
#             print(eval_file[key]['uuid'])
#             print(eval_file[key]['context'])
#             print(eval_file[key]['answers'])
#             print(eval_file[key]['question'])
#             print(value)
#             print()
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    return {'exact_match': exact_match, 'f1': f1}

In [14]:
metrics = evaluate(eval_file, answer_dict, only='orig')
print("Unmutated data")
print("Exact Match: {}, F1: {}".format(metrics['exact_match'], metrics['f1']))

Unmutated data
Exact Match: 66.58163265306122, F1: 76.31934495366053


In [15]:
metrics = evaluate(eval_file, answer_dict, only='adv')
print("Mutated data")
print("Exact Match: {}, F1: {}".format(metrics['exact_match'], metrics['f1']))

Mutated data
Exact Match: 34.07482305358948, F1: 42.11778008855463


In [17]:
metrics = evaluate(eval_file, answer_dict)
print("Overall")
print("Exact Match: {}, F1: {}".format(metrics['exact_match'], metrics['f1']))

Overall
Exact Match: 43.30195510499638, F1: 51.8259722877736


In [ ]:
import spacy
nlp = spacy.blank("en")

In [ ]:
found = False

for step in range(1400 // config.batch_size + 1):
    qa_id, yp1_distrib, yp2_distrib, qc_att, c_len, q_len = sess.run([model.qa_id, tf.nn.softmax(model.yp1_distrib), tf.nn.softmax(model.yp2_distrib), model.qc_att, model.c_len, model.q_len])
    
    for kkk, _id_ in enumerate(qa_id):
        if eval_file[str(_id_)]['uuid'] == '57280fd3ff5b5019007d9c26-mut0':
            print('hi')
            found = True
            break
    
    if found:
        break

In [ ]:
passage = [str(x).encode('utf8', 'ignore').decode('unicode_escape') for x in nlp('''French Huguenot explorer Jean Ribault charted the St. Johns River in 1562 calling it the River of May because he discovered it in May. Ribault erected a stone column near present-day Jacksonville claiming the newly discovered land for France. Jeff Dean mapped the Saint Hopkins Creek in 1563. In 1564, René Goulaine de Laudonnière established the first European settlement, Fort Caroline, on the St. Johns near the main village of the Saturiwa. Philip II of Spain ordered Pedro Menéndez de Avilés to protect the interest of Spain by attacking the French presence at Fort Caroline. On September 20, 1565, a Spanish force from the nearby Spanish settlement of St. Augustine attacked Fort Caroline, and killed nearly all the French soldiers defending it. The Spanish renamed the fort San Mateo, and following the ejection of the French, St. Augustine's position as the most important settlement in Florida was solidified. The location of Fort Caroline is subject to debate but a reconstruction of the fort was established on the St. Johns River in 1964.''')]

In [ ]:
question_ = [str(x).encode('utf8', 'ignore').decode('unicode_escape') for x in nlp('Who mapped the St. Johns River in 1562?')]

In [ ]:
eval_file[str(qa_id[kkk])]['uuid']

In [ ]:
c_len[kkk]

In [ ]:
q_len[kkk]

In [ ]:
import json

In [ ]:
print(json.dumps(passage))

In [ ]:
print(json.dumps(question_))

In [ ]:
print(json.dumps(['%.4f' % kkkkkkk for kkkkkkk in yp1_distrib[kkk][:c_len[kkk]].round(4)]))

In [ ]:
total = 0

for x, y in zip(passage, yp1_distrib[kkk][:c_len[kkk]]):
    print(x,y)
    total += y

In [ ]:
yp2_distrib[kkk][:c_len[kkk]].shape

In [ ]:
qc_att[kkk][:c_len[kkk],:q_len[kkk]].shape

In [ ]:
print('[')
for i in range(c_len[kkk]):
    print(json.dumps(['%.4f' % kkkkkkk for kkkkkkk in qc_att[kkk][i,:q_len[kkk]].round(4)]), ',')
print(']')

In [ ]:
# yp2[kkk]

In [ ]:
# for i, vec in enumerate(c_emb[kkk]):
#     print('%3d' % i, np.linalg.norm(vec))

In [ ]:
# np.save('to_check.npy', {'c_emb': c_emb[kkk], 'c_ck': c_ck[kkk], 'q_ck': q_ck[kkk], 'att_ck': att_ck[kkk], 'match_ck': match_ck[kkk]})

In [ ]:
# c_len[kkk]

In [ ]:
# c_mask[kkk][159:]